Produces cropped TAMSAT geotiffs that are usable in SARRA-O as the full TAMSAT geotiffs and produce the exact same results.

# Imports

In [1]:
import os
import datetime
from datetime import datetime as dt
import requests
import xarray as xr 
from tqdm import tqdm as tqdm

# Functions definition

In [2]:
def download_TAMSAT_day(query_date, save_path):

    query_month = query_date.strftime('%m')
    query_day = query_date.strftime('%d')

    URL_filename = str(query_date.year)+"/"+str(query_month)+"/rfe"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".v3.1.nc"
    URL_full = "http://www.tamsat.org.uk/public_data/data/v3.1/daily/"+URL_filename

    response = requests.get(URL_full)
    save_filename = "TAMSAT_"+URL_filename.replace("/","_")
    open(os.path.join(save_path,save_filename), "wb").write(response.content)

In [3]:
def crop_and_save_TAMSAT_day(query_date, area, selected_area):

    query_month = query_date.strftime('%m')
    query_day = query_date.strftime('%d')
    URL_filename = str(query_date.year)+"/"+str(query_month)+"/rfe"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".v3.1.nc"
    save_filename = "TAMSAT_"+URL_filename.replace("/","_")
    nc_file_content = xr.open_dataset(os.path.join(save_path,save_filename))

    xarray_variable_name = "rfe_filled"

    # cropping
    nc_file_content = nc_file_content.where((nc_file_content.lat < area[selected_area][0])
                            & (nc_file_content.lat > area[selected_area][2])
                            & (nc_file_content.lon > area[selected_area][1])
                            & (nc_file_content.lon < area[selected_area][3])
                        ).dropna(dim='lat', how='all').dropna(dim='lon', how='all')

    bT = nc_file_content[xarray_variable_name]
    bT = bT.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    bT.rio.crs
    bT.rio.write_crs("epsg:4326", inplace=True)
    output_filename = "TAMSAT_v3.1_"+selected_area+"_"+xarray_variable_name+"_"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".tif"
    output_path = "../data/3_output/TAMSAT_v3.1_"+selected_area+"_"+xarray_variable_name+"/"

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    bT.rio.to_raster(os.path.join(output_path,output_filename))

In [4]:
def download_TAMSAT_lastmonth(area, selected_area, save_path):
    query_date = dt.today() - datetime.timedelta(days=8)

    for i in range(query_date.day):

        query_date_loop = query_date - datetime.timedelta(days=i)

        # print("=== downloading TAMSAT for date",query_date_loop.date(),"===")
        download_TAMSAT_day(query_date_loop, save_path)
        crop_and_save_TAMSAT_day(query_date_loop, area, selected_area)

In [5]:
def download_TAMSAT_year(query_year, area, selected_area, save_path):

    end_date = datetime.date(query_year,12,31)
    start_date = datetime.date(query_year,1,1)
    num_days = (end_date-start_date).days
    
    for num_day in tqdm(range(num_days+1)) :

        query_date_loop = datetime.date(query_year,1,1) + datetime.timedelta(days=num_day)

        #print("=== downloading TAMSAT for date",query_date_loop,"===")
        try:
            download_TAMSAT_day(query_date_loop, save_path)
            crop_and_save_TAMSAT_day(query_date_loop, area, selected_area)
        except:
            pass

# Run

In [6]:
# coordinates of area : lat NW, lon NW, lat SE, lon SE
area = {
    'burkina': [16, -6, 9, 3],
    'niger':[23.8, -0.5, 11.3, 15.9],
    'west_africa':[29, -20, 3.5, 26]}

selected_area = "burkina"

save_path = "../data/3_output/"

In [7]:
for year in range(1995,2019):
    query_date = datetime.date(year,1,1) # will run for whole year
    download_TAMSAT_year(year, area, selected_area, save_path)

100%|██████████| 365/365 [05:37<00:00,  1.08it/s]
